In [6]:
#Dependencies
from bs4 import BeautifulSoup
import requests
import json


In [8]:
async def scrapeExtra(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    #printSoupToHtml(soup,'extra.html')

    # Assuming 'soup' is the BeautifulSoup object containing the parsed HTML

    # Find the <script> tag containing ACC.config.productDetails
    script_tag = soup.find('script', {'type': 'text/javascript'})

    if script_tag:
        # Extract the JavaScript content from the <script> tag
        script_content = script_tag.string
        
        # Locate and extract the ACC.config.productDetails object
        start_index = script_content.find('ACC.config.productDetails')
        stop_index = script_content.find('ACC.config.currentCity')
 
        if start_index != -1 and stop_index != -1:
            product_details = script_content[start_index:stop_index]
            product_details = product_details.replace('ACC.config.productDetails = ', '').strip()
            product_details = product_details[:-1]
            product_details = json.loads(product_details)
            #printSoupToJson(product_details)
            #print(product_details)
            name_local = product_details.get('name', 'N/A')
            name_global = product_details.get('nameEn', 'N/A')
            isAvailable = product_details.get('stock', {}).get('stockLevel', 'N/A')>0

            total_price = product_details.get('price', {}).get('formattedValue', 'N/A')
            discount = product_details.get('percentageDiscount', {}).get('value', 'N/A')
            PriceAfterDiscount = float(total_price) - (float(total_price) * discount / 100)
            description = product_details.get('descriptionEn', 'N/A')
            #arabic description
            rating = product_details.get('rating', 'N/A')
            image= product_details.get('highlights')
            soup = BeautifulSoup(image, 'html.parser')
            image = soup.find_all('img')
            images= [ img['src'] for img in image]
              # Print or use the extracted data
            print('name_local:', name_local)
            print('name_global:', name_global)
            print('isAvailable:', isAvailable)
            print('Price:', PriceAfterDiscount)
            print('Description:', description)
            print('Rating:', rating)
            print('Images:', images)
            


In [9]:
#Test
url = 'https://www.extra.com/ar-sa/mobiles-tablets/mobiles/smartphone/samsung-galaxy-z-fold-5-5g-512gb-icy-blue/p/100343500'
print(await scrapeExtra(url))

name_local: سامسونج جالكسي زد فولد 5، ، 5 جي ، 512 جيجا، ازرق
name_global: Samsung Galaxy Z Fold 5, 5G, 512GB, Icy Blue
isAvailable: False
Price: 5799.3362
Description: SM-F946BLBGMEA--Samsung Galaxy Z Fold 5, 5G, 7.6-inch, Dynamic AMOLED 2X, Dual SIM, 512GB, Icy Blue, Snapdragon 8 Gen 2, Android 13, 12GB RAM, 50MP+12MP+10MP Rear Camera, 10MP+4MP Front Camera, 4400mAh Battery
Rating: 4.35
Images: ['https://aurora.a.bigcontent.io/v1/static/100343500_01', 'https://aurora.a.bigcontent.io/v1/static/100343500_02', 'https://aurora.a.bigcontent.io/v1/static/100343500_03', 'https://aurora.a.bigcontent.io/v1/static/100343500_04', 'https://aurora.a.bigcontent.io/v1/static/100343500_05', 'https://aurora.a.bigcontent.io/v1/static/100343500_06', 'https://aurora.a.bigcontent.io/v1/static/100343500_07', 'https://aurora.a.bigcontent.io/v1/static/100343500_08', 'https://aurora.a.bigcontent.io/v1/static/100343500_09', 'https://aurora.a.bigcontent.io/v1/static/100343500_10']
None


In [7]:
#Dependencies
from bs4 import BeautifulSoup
import requests
import json
from utils import getStringBetweenTwoWords, extract_and_stringify_object , printJsonToFile
from Product import ProductDetailDTO

async def scrape_extra_full(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the <script> tag containing ACC.config.productDetails
    script_tag = soup.find('script', {'type': 'text/javascript'})

    if script_tag:
        # Extract the JavaScript content from the <script> tag
        script_content = script_tag.string
        
        # Locate and extract the ACC.config.productDetails object
        start_index = script_content.find('ACC.config.productDetails')
        stop_index = script_content.find('ACC.config.currentCity')

        
        if start_index != -1 and stop_index != -1:
            product_details = script_content[start_index:stop_index]
            product_details = product_details.replace('ACC.config.productDetails = ', '').strip()
            product_details = product_details[:-1]
            product_details = json.loads(product_details)
            printJsonToFile(product_details,'extra.json')

            product_details = json.loads(open('extra.json').read())

            name_global = product_details.get('nameEn', 'N/A')
            name_local = product_details.get('name', 'N/A')
            isAvailable = product_details.get('stock', {}).get('stockLevel', 'N/A')>0
            total_price = product_details.get('price', {}).get('formattedValue', 'N/A')
            discount = product_details.get('percentageDiscount', {}).get('value', 'N/A')
            PriceAfterDiscount = float(total_price) - (float(total_price) * discount / 100)
            description = product_details.get('descriptionEn', 'N/A')
            rating = product_details.get('rating', 'N/A')
            image= product_details.get('highlights')
            soup = BeautifulSoup(image, 'html.parser')
            image = soup.find_all('img')
            images= [ img['src'] for img in image]
            item_data = ProductDetailDTO(name_Global=name_global,name_Local=name_local,is_available=isAvailable, price=PriceAfterDiscount, rating=rating, description_Global=description,images=images,productlink1=url)
            return item_data
x=await scrape_extra_full('https://www.extra.com/ar-sa/mobiles-tablets/mobiles/smartphone/samsung-galaxy-z-fold-5-5g-512gb-icy-blue/p/100343500')   
print("name_global:",x.name_Global)
print("isAvailable:",x.is_available)    
print("price:",x.price)
print("rating:",x.rating)
print("description:",x.description_Global)
print("images:",x.images)




name_global: Samsung Galaxy Z Fold 5, 5G, 512GB, Icy Blue
isAvailable: False
price: 5799.3362
rating: 4.35
description: SM-F946BLBGMEA--Samsung Galaxy Z Fold 5, 5G, 7.6-inch, Dynamic AMOLED 2X, Dual SIM, 512GB, Icy Blue, Snapdragon 8 Gen 2, Android 13, 12GB RAM, 50MP+12MP+10MP Rear Camera, 10MP+4MP Front Camera, 4400mAh Battery
images: ['https://aurora.a.bigcontent.io/v1/static/100343500_01', 'https://aurora.a.bigcontent.io/v1/static/100343500_02', 'https://aurora.a.bigcontent.io/v1/static/100343500_03', 'https://aurora.a.bigcontent.io/v1/static/100343500_04', 'https://aurora.a.bigcontent.io/v1/static/100343500_05', 'https://aurora.a.bigcontent.io/v1/static/100343500_06', 'https://aurora.a.bigcontent.io/v1/static/100343500_07', 'https://aurora.a.bigcontent.io/v1/static/100343500_08', 'https://aurora.a.bigcontent.io/v1/static/100343500_09', 'https://aurora.a.bigcontent.io/v1/static/100343500_10']
